In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from typing import *

import pandas as pd
import plotly.express as px

from spot.utils import cst, proj_root, run_long_task, tqdm

os.chdir(proj_root())

datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"

In [2]:
# experiment configurations

import torch

from spot.data import SrcDataset, TokenizedSrc, get_model_name, load_src_datasets
from spot.model import CtxArgs, DecodingArgs, ModelSPOT, ModelTrainingArgs, ModelWrapper

quicktest = False
drop_comments = True
data_reduction = 1
train_R1 = True
max_tokens_per_file = 4096


# TODO: try 2K, 1K, 1K model
ctx_args = CtxArgs(
    ctx_size=max_tokens_per_file,
    left_margin=512,
    right_margin=512,
    types_in_ctx=False,
)

dec_args = DecodingArgs(
    sampling_max_tokens=4 * max_tokens_per_file,
    ctx_args=ctx_args,
    max_workers=20,
)


r0_model_name = get_model_name(
    drop_comments=drop_comments,
    ctx_args=ctx_args,
    data_reduction=data_reduction,
    quicktest=quicktest,
)

src_datasets = load_src_datasets(
    datadir,
    drop_comments=drop_comments,
    spot_round=0,
    data_reduction=data_reduction,
    repos_root=datadir / "SPOT-data/repos",
    quicktest=quicktest,
)

for n in ["train", "valid", "test"]:
    print("Filtering files for", n)
    tkns = max_tokens_per_file
    src_datasets[n].filter_files(min_labels_per_file=3, min_tokens_per_file=0, max_labels_per_file=10000, max_tokens_per_file=1000000)


Filtering files for train
labels_kept_after_filtering: 0.95591
tkns_kept_after_filtering: 0.87797
Filtering files for valid
labels_kept_after_filtering: 0.9611
tkns_kept_after_filtering: 0.9049
Filtering files for test
labels_kept_after_filtering: 0.95657
tkns_kept_after_filtering: 0.89668


In [3]:
# train the model
from spot.train import train_spot_model

train_args = ModelTrainingArgs(
    train_max_tokens=max_tokens_per_file,
    eval_max_tokens=2 * max_tokens_per_file,
    max_epochs=3,
)

r0_wrapper, r0_extra = train_spot_model(
    src_datasets,
    r0_model_name,
    dec_args=dec_args,
    train_args=train_args,
    record_batches=train_R1,
    quicktest=quicktest,
    use_small_model=False,
)


/home/jiayi/Projects/SPOT/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


chunk_srcs_per_file:   0%|          | 0/726 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/856 [00:00<?, ?it/s]

chunk_srcs_per_file:   0%|          | 0/619 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/699 [00:00<?, ?it/s]

...

type_check_src:   0%|          | 0/10009 [00:00<?, ?it/s]

type_check_success_ratio: 0.9985
feedbacks_per_file:
   mean: 1.191
   median: 0
   min: 0
   max: 243


feedbacks_to_tokenized_src:   0%|          | 0/10009 [00:00<?, ?it/s]

Pushover: (Failed: Generating R1 datasets.) Code 1:
<<
from typing import *
import json
 
from feedgen.feed import FeedGenerator
from flask import request, redirect, Response

from bozen.butil import pr, prn, dpr, form, htmlEsc
from bozen import FormDoc, MonDoc, BzDate, BzDateTime
from bozen import paginate

import config
from allpages import app, jinjaEnv
import ht
from userdb import User
import models
from permission import needUser, currentUserName

import messlist
   

@app.route('/listFollowing/<id>')
def listFollowing(id):
    user = User.getDoc(id)
    ai = models.getAccountInfo(id)
    count = len(ai.following_ids)
    pag = paginate.Paginator(count)
    
    tem = jinjaEnv.get_template("listFollowing.html")
    h = tem.render(
        id = id,
        user = user,
        ai = ai,
        count = count,
        pag = pag,
        table = followingTableH(ai, pag),
    )
    return h
 
def followingTableH(ai: models.AccountInfo, pag: paginate.Paginator) -> str:
    h = USER_INFO

--- Logging error ---
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/jiayi/anaconda3/envs/py3.10/lib/python3.10/concurrent/futures/process.py", line 243, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/home/jiayi/anaconda3/envs/py3.10/lib/python3.10/concurrent/futures/process.py", line 202, in _process_chunk
    return [fn(*args) for args in chunk]
  File "/home/jiayi/anaconda3/envs/py3.10/lib/python3.10/concurrent/futures/process.py", line 202, in <listcomp>
    return [fn(*args) for args in chunk]
  File "/home/jiayi/Projects/SPOT/src/spot/data.py", line 241, in feedbacks_to_tokenized_src
    assert m_code == current_code, "Code 1:\n<<{}>>\nCode 2:\n<<{}>>".format(
AssertionError: Code 1:
<<
from typing import *
import json
 
from feedgen.feed import FeedGenerator
from flask import request, redirect, Response

from bozen.butil import pr, prn, dpr, form, htmlEsc
from bozen import FormDoc, MonDoc,

In [ ]:
from spot.train import R1_srcs_from_extra

r0_wrapper = ModelWrapper.from_pretrained(
    datadir / f"checkpoints/lit-saved/{r0_model_name}"
)

chunks: dict[str, ChunkedDataset] = {}
with run_long_task("Preparing chunked datasets", notify=False):
    for n in ["valid", "test", "train"]:
        src = src_datasets[n]
        chunks[n] = src.to_chunks(r0_wrapper.tokenizer, dec_args.ctx_args)

r0_extra = pickle_load(datadir / "checkpoints/lit-running" / r0_model_name / "extra.pickle")
ckpt_dir = datadir / "checkpoints/lit-running" / r0_model_name / "models"

R1_srcs_from_extra(
    r0_wrapper,
    src_datasets,
    chunks,
    r0_extra,
    ckpt_dir,
    ckpt_interval=350,
)
datadir / "checkpoints/lit-running" / r0_model_name / "extra.pickle"

In [10]:
# load trained model
from spot.utils import pickle_load, pickle_dump

r0_wrapper = ModelWrapper.from_pretrained(
    datadir / f"checkpoints/lit-saved/{r0_model_name}"
)
if train_R1:
    r0_extra = pickle_load(datadir / f"checkpoints/lit-saved/{r0_model_name}/extra.pkl")
    r1_src_datasets: dict[str, SrcDataset] = r0_extra["R1-src_datasets"]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
r0_wrapper.to(device)
r0_wrapper.args.do_sample = False
print(r0_wrapper.args)


DecodingArgs(ctx_args=CtxArgs(left=512, window=3072, right=512), sampling_max_tokens=16384, max_workers=20, max_tokens_per_type=10, do_sample=False, top_p=0.9, num_beams=None)


In [ ]:
# checking mypy feedbacks

from spot.type_env import MypyFeedback
from spot.utils import groupby, pretty_print_accuracies


def ground_truth_feedback(dataset: SrcDataset):
    preds = {
        f: {i: str(t) for i, t in enumerate(src.types)}
        for f, src in dataset.file2src().items()
    }
    return dataset.add_type_checker_feedback(
        r0_wrapper.tokenizer, preds, max_workers=20, tqdm_args={}
    )


def show_feedback_stats(new_dataset: SrcDataset):
    fb_list: list[list[MypyFeedback]] = new_dataset.extra_stats["mypy_feedbacks"]
    stats = {}
    stats["total_feedbacks"] = sum(len(l) for l in fb_list)
    error_code_counter = Counter[str]()
    for l in fb_list:
        for fb in l:
            error_code_counter[fb.error_code] += 1
    stats["top_feedbacks"] = dict(error_code_counter.most_common(10))
    pretty_print_accuracies(stats)
    df = pd.DataFrame(error_code_counter.most_common(), columns=["error_code", "count"])
    display(px.bar(df, x="error_code", y="count", title="Error code frequencies"))
    fdbk_srcs = [(f, src) for src, fs in zip(new_dataset.all_srcs, fb_list) for f in fs]
    error_groups = groupby(fdbk_srcs, lambda x: x[0].error_code)
    return error_groups


# new_src = ground_truth_feedback(src_datasets["test"])
if train_R1:
    error_groups = show_feedback_stats(r1_src_datasets["test"])


total_feedbacks: 13
top_feedbacks:
   attr-defined: 4
   return-value: 3
   type-arg: 2
   return: 1
   assignment: 1
   name-defined: 1
   syntax: 1


In [ ]:
from spot.visualization import visualize_sequence
from spot.utils import seq_flatten


def add_line_numbers(code: str):
    lines = code.split("\n")
    return "\n".join(f"{i+1:3d}|  {l}" for i, l in enumerate(lines))


if train_R1:
    to_display = []
    for xs in seq_flatten(error_groups.values()):
        to_display.append(
            f"feedback: {xs[0]}\n"
            + "=========code=========\n"
            + add_line_numbers(xs[1].origin_code)
        )
    display(visualize_sequence(to_display))


In [11]:
# model evaluation

import ipywidgets as widgets
import plotly.express as px

from spot.data import R1_srcs_from_preds, load_src_datasets, pretty_print_accuracies
from spot.utils import PickleCache, assert_eq
from spot.visualization import visualize_sequence


def eval_model_helper(
    r0_wrapper: ModelWrapper,
    r1_wrapper: Optional[ModelWrapper],
    eval_cache: PickleCache,
    r0_srcs: SrcDataset,
    reeval=False,
):
    if reeval:
        eval_cache.clear()
    r0_accs, r0_chunks, r0_preds = eval_cache.cached(
        "r0_eval.pkl",
        lambda: r0_wrapper.eval_on_dataset(r0_srcs, tqdm_args={"leave": False}),
    )
    if r1_wrapper is None:
        return r0_accs

    r1_srcs = eval_cache.cached(
        "r1_srcs.pkl",
        lambda: R1_srcs_from_preds(
            r1_wrapper.tokenizer,
            r0_srcs,
            r0_chunks.chunks_info,
            r0_chunks.files,
            r0_preds,
            max_workers=r0_wrapper.args.max_workers,
        ),
    )

    r1_accs, _, _ = eval_cache.cached(
        "r1_eval.pkl",
        lambda: r1_wrapper.eval_on_dataset(r1_srcs, tqdm_args={"leave": False}),
    )

    return (r0_accs, r1_accs)


def evaluate_model(
    r0_wrapper: ModelWrapper,
    r1_wrapper: Optional[ModelWrapper],
    model_name: str,
    r0_srcs: SrcDataset,
    size_factors=[1],
    reeval=False,
):

    acc_series = []
    accs_seq = []
    if r1_wrapper is not None:
        assert_eq(r0_wrapper.args, r1_wrapper.args)
    with run_long_task(f"Evaluate accuracy vs ctx_size: {model_name}"):
        for factor in size_factors:
            wrapper0 = r0_wrapper.scale_ctx_size(factor)
            wrapper1 = (
                r1_wrapper.scale_ctx_size(factor) if r1_wrapper is not None else None
            )
            eval_cache = PickleCache(
                datadir / f"checkpoints/lit-saved/{model_name}/eval-ctx_size={factor}"
            )
            accs = eval_model_helper(
                wrapper0, wrapper1, eval_cache, r0_srcs, reeval=reeval
            )
            if isinstance(accs, tuple):
                acc_series.append(accs[1])
                accs_seq.extend([("R0", accs[0]), ("R1", accs[1])])
            else:
                accs_seq.extend([("R0", accs)])
                acc_series.append(accs)

    def print_acc(i, expand):
        m, a = accs_seq[i]
        print(f"model={m}, ctx_size_factor={factor}")
        print(f"ctx_args: {wrapper0.args.ctx_args}")
        pretty_print_accuracies(a, max_show_level=100 if expand else 0)

    display(widgets.interactive(print_acc, i=(0, len(accs_seq) - 1), expand=False))

    if len(size_factors) > 1:
        acc_df = pd.DataFrame(
            {
                "ctx_size": size_factors,
                "partial_acc": [x["partial_acc"] for x in acc_series],
                "full_acc": [x["full_acc"] for x in acc_series],
                "full_acc_strict": [x["full_acc_strict"] for x in acc_series],
            }
        )
        display(
            px.line(
                acc_df,
                x="ctx_size",
                y=["partial_acc", "full_acc", "full_acc_strict"],
                title=model_name,
            )
        )
    return acc_series

In [7]:

evaluate_model(
    r0_wrapper,
    None,
    r0_model_name,
    src_datasets["test"],
    reeval=True,
)
None


chunk_srcs_per_file:   0%|          | 0/619 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/699 [00:00<?, ?it/s]

predict:   0%|          | 0/699 [00:00<?, ?it/s]

[PickleCache] Saving to cache: '/mnt/data0/jiayi/checkpoints/lit-saved/SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments/eval-ctx_size=1/r0_eval.pkl'
Pushover: (Finished: 'Evaluate accuracy vs ctx_size: SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments'.) Time taken: 250.3s


interactive(children=(IntSlider(value=0, description='i', max=0), Checkbox(value=False, description='expand'),…

In [9]:
r0_wrapper.args.ctx_args = CtxArgs(
    max_tokens_per_file,
    1024 * 2,
    1024
)
evaluate_model(
    r0_wrapper,
    None,
    r0_model_name,
    src_datasets["test"],
    reeval=True,
)
None

chunk_srcs_per_file:   0%|          | 0/619 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/800 [00:00<?, ?it/s]

predict:   0%|          | 0/800 [00:00<?, ?it/s]

[PickleCache] Saving to cache: '/mnt/data0/jiayi/checkpoints/lit-saved/SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments/eval-ctx_size=1/r0_eval.pkl'
Pushover: (Finished: 'Evaluate accuracy vs ctx_size: SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments'.) Time taken: 323.0s


interactive(children=(IntSlider(value=0, description='i', max=0), Checkbox(value=False, description='expand'),…

In [11]:
r0_wrapper.args.ctx_args = CtxArgs(
    max_tokens_per_file,
    1024 * 2,
    1024 + 512,
)
r0_wrapper.args.num_beams = 4
evaluate_model(
    r0_wrapper,
    None,
    r0_model_name,
    src_datasets["test"],
    reeval=True,
)
None

chunk_srcs_per_file:   0%|          | 0/619 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/932 [00:00<?, ?it/s]

predict:   0%|          | 0/932 [00:00<?, ?it/s]

[PickleCache] Saving to cache: '/mnt/data0/jiayi/checkpoints/lit-saved/SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments/eval-ctx_size=1/r0_eval.pkl'
Pushover: (Finished: 'Evaluate accuracy vs ctx_size: SPOT-model-per_file-R0-(512, 3072, 512)-drop_comments'.) Time taken: 728.6s


interactive(children=(IntSlider(value=0, description='i', max=0), Checkbox(value=False, description='expand'),…

In [12]:
r0_wrapper.args.ctx_args = CtxArgs(
    max_tokens_per_file,
    1024 * 2,
    1024,
)
r0_wrapper.args.num_beams = 16
r0_wrapper.args.sampling_max_tokens = max_tokens_per_file
evaluate_model(
    r0_wrapper,
    None,
    r0_model_name,
    src_datasets["test"],
    reeval=True,
)
None

chunk_srcs_per_file:   0%|          | 0/619 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/800 [00:00<?, ?it/s]

predict:   0%|          | 0/800 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# R1 experiment configurations

import torch

from spot.data import ChunkedDataset, SrcDataset, get_dataset_name, get_model_name
from spot.model import CtxArgs, DecodingArgs, ModelSPOT, ModelTrainingArgs, ModelWrapper

r1_model_name = get_model_name(
    spot_round=1,
    drop_comments=drop_comments,
    ctx_args=ctx_args,
    data_reduction=data_reduction,
    quicktest=quicktest,
)


In [ ]:
train_args.max_epochs = 3

r1_wrapper, r1_extra = train_spot_model(
    r1_src_datasets,
    r1_model_name,
    dec_args=dec_args,
    train_args=train_args,
    record_batches=False,
    quicktest=quicktest,
)


chunk_srcs_per_file:   0%|          | 0/4 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/4 [00:00<?, ?it/s]

chunk_srcs_per_file:   0%|          | 0/7 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/7 [00:00<?, ?it/s]

chunk_srcs_per_file:   0%|          | 0/7 [00:00<?, ?it/s]

...

Pushover: (Finished: 'Training quicktest-SPOT-model-per_file-R1-(512, 3072, 512)-drop_comments'.) Time taken: 42.8s


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1,2]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       valid/loss           1.3097461462020874
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▃▃▃▆▆▆█
trainer/global_step,▁▂▃▃▄▅▆▆▇█
valid/loss,██▂▂▂▁▁▁▁▁
epoch,3
trainer/global_step,9
valid/loss,1.30975


In [ ]:
# load trained model and evaluate

r1_wrapper = ModelWrapper.from_pretrained(
    datadir / f"checkpoints/lit-saved/{r1_model_name}"
)

r1_wrapper.to(device)

evaluate_model(
    r0_wrapper,
    r1_wrapper,
    r1_model_name,
    src_datasets["test"],
    reeval=False,
)
None


chunk_srcs_per_file:   0%|          | 0/7 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/7 [00:00<?, ?it/s]

predict:   0%|          | 0/7 [00:00<?, ?it/s]

[PickleCache] Saving to cache: '/mnt/data0/jiayi/checkpoints/lit-saved/quicktest-SPOT-model-per_file-R1-(512, 3072, 512)-drop_comments/eval-ctx_size=1/r0_eval.pkl'


type_check_src:   0%|          | 0/7 [00:00<?, ?it/s]

...

chunk_srcs_per_file:   0%|          | 0/7 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/7 [00:00<?, ?it/s]

predict:   0%|          | 0/7 [00:00<?, ?it/s]

[PickleCache] Saving to cache: '/mnt/data0/jiayi/checkpoints/lit-saved/quicktest-SPOT-model-per_file-R1-(512, 3072, 512)-drop_comments/eval-ctx_size=1/r1_eval.pkl'
Pushover: (Finished: 'Evaluate accuracy vs ctx_size: quicktest-SPOT-model-per_file-R1-(512, 3072, 512)-drop_comments'.) Time taken: 13.0s


interactive(children=(IntSlider(value=0, description='i', max=1), Checkbox(value=False, description='expand'),…

In [ ]:
from IPython.display import display

from spot.visualization import visualize_batch, visualize_code_sequence


def visualize_preds_code(
    wrapper: ModelWrapper,
    src_dataset: SrcDataset,
    n_visual_exs: int = 16,
):
    _, visual_data, visual_preds = wrapper.eval_on_dataset(
        src_dataset[:n_visual_exs], tqdm_args={"leave": False}
    )

    display(
        visualize_code_sequence(
            [
                visualize_batch(
                    visual_data,
                    i,
                    visual_preds,
                )
                for i in range(min(n_visual_exs, len(visual_preds)))
            ]
        )
    )

visualize_preds_code(r1_wrapper, r1_src_datasets["test"], n_visual_exs=50)


chunk_srcs_per_file:   0%|          | 0/7 [00:00<?, ?it/s]

verify_labels:   0%|          | 0/7 [00:00<?, ?it/s]

predict:   0%|          | 0/7 [00:00<?, ?it/s]